<a href="https://colab.research.google.com/github/Thiagokiam/CNN_Models_DeepLearning/blob/main/Save_DataSet_TensorData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tratamento dos dados
import cv2 as cv
import time, os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# Implementação e treinamento da rede
import torch
from torch import nn, optim

# Carregamento de Dados
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

# Plots e análises
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def ConversaoDataFrame (diretorio_imagens, diretorio_dados, diretorio_perneabilidade):
  # Diretórios das imagens e dos dados
  diretorio_imagens = diretorio_imagens
  diretorio_dados = diretorio_dados
  diretorio_perneabilidade = diretorio_perneabilidade

  arquivos = sorted(os.listdir(diretorio_imagens))
  dados = sorted(os.listdir(diretorio_dados))

  dados_permeabilidade = pd.read_excel(diretorio_perneabilidade, sheet_name = 'Planilha1')
  dados_permeabilidade['Permeabilidade (mD)'][0]

  # Coletando cada uma das imagens
  imagem = []
  for i in np.arange(len(arquivos)):
    diretorio = diretorio_imagens + str(arquivos[i]) + '/'
    lista_imagens = sorted(os.listdir(diretorio))
    img = []
    for j in np.arange(len(lista_imagens)):
      diretorio_imgem = diretorio + lista_imagens[j]
      img.append(cv.imread(diretorio_imgem, cv.IMREAD_GRAYSCALE))
    imagem.append(img)

  amostras = len(imagem)        # Quantidade de amostras
  n_imagens = len(imagem[0])    # Quantidade de imagens que cada amostra possui

  # Coletando cada um dos Threshould's
  dados_threshould = []
  nome_amostra = []
  for i in np.arange(amostras):
    dir_dados = diretorio_dados + str(dados[i])
    arqu = pd.read_csv(dir_dados, sep = ':').T.reset_index().drop('index', axis = 1).T
    treshould = []
    for j in np.arange(n_imagens):
      treshould.append(int(arqu[1][j][1:-4]))
      nome_amostra.append(dados[i][:3])
    dados_threshould.append(treshould)


  # Obtendo os histogramas, a escala de cinza e a multiplicação
  escala_cinza = np.arange(255, -1, -1)
  histogramas = []
  multiplicacao_hist_cinza = []
  soma_multiplicacao = []

  for i in np.arange(amostras):
    threshould = dados_threshould[i]
    image = imagem[i]
    histograma = []
    multiplicacao = []
    soma = []
    for j in np.arange(n_imagens):
      histg = cv.calcHist([image[j]], [0], None, [256], [0,256])
      mult = histg[:threshould[j]].T[0]*escala_cinza[:threshould[j]]
      histograma.append(histg)
      multiplicacao.append(mult)
      soma.append(sum(mult))
    histogramas.append(histograma)
    multiplicacao_hist_cinza.append(multiplicacao)
    soma_multiplicacao.append(soma)


  # Calculando a permeabilidade em cada Slice
  permeabilidade_slice = []
  for i in np.arange(amostras):
    k_slice = []
    for j in np.arange(n_imagens):
      soma_amostra = sum(soma_multiplicacao[i])/n_imagens
      k_gas = dados_permeabilidade['Permeabilidade (mD)'][i]
      soma_slice = soma_multiplicacao[i][j]
      k_slice.append(k_gas*soma_slice/soma_amostra)
    permeabilidade_slice.append(k_slice)

  # Criando DataFrame com os dados da imagem e permeabilidade
  lista_imagem = []
  permeabilidade_input = []
  for i in np.arange(amostras):
    for j in np.arange(n_imagens):
      #reshape = imagem[i][j].reshape(1, pixel_imagem*pixel_imagem)
      lista_imagem.append(imagem[i][j])
      permeabilidade_input.append(permeabilidade_slice[i][j])

  dados_entrada = pd.DataFrame({'Amostra': nome_amostra,
                                'Imagem': lista_imagem,
                                'Permeabilidade': permeabilidade_input})

  return dados_entrada

In [ ]:
diretorio_perneabilidade = '/content/drive/MyDrive/Projeto_CNN/Dados_petrofisicos_tentativa_-nova.xlsx', sheetname = 'Teste'

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<ipython-input-1-eeea75551f2d>, line 1)

In [ ]:
diretorio_imagens = '/content/drive/MyDrive/Projeto_CNN/Imagens/'
diretorio_dados = '/content/drive/MyDrive/Projeto_CNN/Dados_Threshould/'
diretorio_perneabilidade = '/content/drive/MyDrive/Projeto_CNN/Dados_petrofisicos_tentativa_-nova.xlsx', sheetname = 'Teste'
df = ConversaoDataFrame(diretorio_imagens,diretorio_dados, diretorio_perneabilidade)


In [ ]:
df.to_excel('/content/drive/MyDrive/ModelCNN/Dados_petrofisicos.xlsx')

In [ ]:
len(df["Imagem"][0])

450

In [ ]:
# Criando tensores
imagem_tensor = []
perm=np.log10(df['Permeabilidade']*1000)
permeabilidade_tns = torch.from_numpy(np.array(perm).astype(np.float16))
for i in np.arange(len(df)):
  con_tns = torch.Tensor(df['Imagem'][i])
  img_tns = con_tns.view(1, con_tns.size(0), con_tns.size(1))
  imagem_tensor.append(img_tns)

print(len(img_tns))
print(permeabilidade_tns[1])
print(df['Permeabilidade'][5])

In [ ]:
# Aleatorizando os atributos e rótulos
X_image, y_perm = imagem_tensor, permeabilidade_tns
torch.save(X_image, '/content/drive/MyDrive/Projeto_CNN/Tensores/atributos_random_tensor_n_aleatorio.pt')
torch.save(y_perm, '/content/drive/MyDrive/Projeto_CNN/Tensores/rotulos_random_tensor_n_aleatorio.pt')